In [300]:
import pandas as pd
import numpy as np
import re
import datetime
from typing import Any, Union
from exeptions import NoneParameter

In [301]:
tytle_table = pd.read_excel('Экспресс-отчет КЦ-2 КС Елизаветинская Северного ЛПУМГ цех.xlsx', sheet_name='Титульный лист', engine='openpyxl')

In [303]:
def _search_cells(searching_row: str,
                     end_row: Union[int, None] = None,
                     start_row: int = 0,
                     check_column: int = 0) -> int:
    """Поиск ячеек на титульном листе"""
    for index, cell in enumerate(tytle_table.iloc[start_row:end_row,
                                                  check_column]):
        if re.search(searching_row, str(cell)) is not None:
            return index
        
def _search_eq_and_spec(header_name:str, check_column:int) -> None:
    """Поиск оборудования или специалистов с помощением их в список"""
    eq_or_spec_list = []
    try:
        header_row = _search_cells(header_name,  check_column=check_column) + 1
    except TypeError:
        print("На титульном листе удалены сведения о Зав.№ оборудования/ФИО специалистов")
        raise TypeError
    while not pd.isnull(tytle_table.iloc[header_row, check_column]):        
        eq_or_spec_list.append(tytle_table.iloc[header_row, check_column])
        header_row += 1
    return eq_or_spec_list

def _check_none_parameter(*parameters: Any) -> None:
    """Проверка, что все обязательные поля заполнены"""
    for parameter in parameters:
        if not parameter:
            raise NoneParameter("На Титульном листе экспресс-отчета отсутствуют обязательные параметры")
            
def _normilize_parameters(vtd_obj_name: str,
                          vtd_obj_type: str,
                          pipeline_category: str, 
                          pipeline_name: str) -> Any:
    """Приведение параметров к нормальному виду"""
    def _remove_none_value(pipeline_name: str) -> Union[str, None]:
        if not re.fullmatch(r'\w', pipeline_name):
            pipeline_name = None
        return pipeline_name
    def _correcting_types(vtd_obj_type: str) -> str:
        if re.search(r'[пП]лощад|[цЦ]ех', vtd_obj_type) and re.search(r'[шШ]лейф|[уУ]зел|[уУ]зла', vtd_obj_type):
            vtd_obj_type = "Технологические трубопроводы"
        elif re.search(r'[пП]лощад|[цЦ]ех', vtd_obj_type):
            vtd_obj_type = "Внутриплощадочные технологические трубопроводы"
        elif re.search(r'[шШ]лейф', vtd_obj_type):
            vtd_obj_type = "Технологические трубопроводы подключающих шлейфов"
        elif re.search(r'[уУ]зел|[уУ]зла', vtd_obj_type):
            vtd_obj_type = "Технологические трубопроводы узла подключения"
        elif re.search(r'[мМ]агистрал', vtd_obj_type):
            vtd_obj_type = "Магистральный трубопровод"
        else:
            vtd_obj_type = "Технологические трубопроводы"
        return vtd_obj_type

    def _parse_vtd_obj_name(vtd_obj_name: str) -> Any:
        kc_number = int(re.search(r'КЦ-\d',
                              vtd_obj_name)[0].split('КЦ-')[1])
        lpumg_name = re.search(r'\s\w+\sЛПУМГ', 
                               vtd_obj_name)[0].split('ЛПУМГ')[0]
        match = re.search(r'КЦ-\d+',
                      vtd_obj_name)
        if match:
            kc_number = int(match[0].split('КЦ-')[1])
        else:
            kc_number = None
        match = re.search(r'КС-\d+',
                          vtd_obj_name)
        if match:
            ks_number = int(match[0].split('КС-')[1])
        else:
            ks_number = None
        match = re.search(r'КС\S*\s(?!КЦ|\w+ого|\w+ое)\S+',
                          vtd_obj_name)
        if match:
            ks_name = re.split(r'КС\S*\s', match[0])[1].replace('"', '')
        else:
            ks_name = None
        lpumg_name = re.search(r'\s\w+\sЛПУМГ', vtd_obj_name)[0].split(' ЛПУМГ')[0]
        return kc_number, ks_number, ks_name, lpumg_name
    
    pipeline_name = _remove_none_value(pipeline_name)
    pipeline_category_list = pipeline_category.replace(" ", "").split(",")
    vtd_obj_type = _correcting_types(vtd_obj_type)
    parse_vtd_obj_parameters = _parse_vtd_obj_name(vtd_obj_name)
       
    return pipeline_category_list, pipeline_name, vtd_obj_type, *parse_vtd_obj_parameters

def tytle_table_parse() -> Any:
    try:
        # Наименование заказчика
        client_name = tytle_table.iloc[_search_cells(r'Наименование общества'), 1]
        #Наименование объекта контроля
        vtd_obj_name = tytle_table.iloc[_search_cells(r'Наименование об\wекта'), 1]
        # Наименование газопровода/трубопровода
        pipeline_name = tytle_table.iloc[_search_cells(r'Наименование \w+провода'), 1]
        # Вид объекта контроля
        vtd_obj_type = tytle_table.iloc[_search_cells(r'Вид об\wекта'), 1]
        # Даты начала и окончания работ
        vtd_start_date = tytle_table.iloc[_search_cells(r'Дата начала'), 1]
        vtd_end_date = tytle_table.iloc[_search_cells(r'Дата окончания'), 1]
        # Категория трубопровода
        pipeline_category = tytle_table.iloc[_search_cells(r'Категория',  check_column=5), 6]
        # Номер и дата договора/письма
        try:
            contract_name_and_date = tytle_table.iloc[_search_cells(r'договор|письм\w',  check_column=3) + 1, 3]
        except TypeError:
            print("На титульном листе удалены сведения о номере договора/письма")
            raise TypeError  
        # Перечень заводских номеров оборудования
        equipment_numbers_list = _search_eq_and_spec(r'Зав', 2)
        # Перечень специалистов
        specialists_list = _search_eq_and_spec(r'Ф\sИ\sО', 1)
        #
        _check_none_parameter(client_name,
                      vtd_obj_name,
                      vtd_obj_type,
                      vtd_start_date,
                      vtd_end_date,
                      pipeline_category,
                      equipment_numbers_list,
                      specialists_list)
    except ValueError:
        print("На титульном листе удалены или переименованы необходимые строки")
        raise ValueError
    normalize_parameters = _normilize_parameters(vtd_obj_name, 
                                                 vtd_obj_type, 
                                                 pipeline_category, 
                                                 pipeline_name)
    return_parameters = (client_name,
                         *normalize_parameters,
                         vtd_start_date, 
                         vtd_end_date,
                         equipment_numbers_list,
                         specialists_list                     
                        )
    return return_parameters


In [304]:
if __name__ == '__main__':
    print(tytle_table_parse())

('ООО "Газпром трансгаз Санкт-Петербург"', ['В'], None, 'Внутриплощадочные технологические трубопроводы', 2, None, 'Елизаветинская', ' Северного', datetime.datetime(2022, 6, 22, 0, 0), datetime.datetime(2022, 6, 28, 0, 0), [9, 140555], ['Соловьев Олег Романович'])
